# E-bay car sales analysis
### Hemanth Soni, July 2020

---

The goal for this project is to clean and analyze a subset of eBay car sales data. The [original database](https://www.kaggle.com/orgesleka/used-cars-database/data) was uploaded to Kaggle, but [Dataquest](https://dataquest.io) has created a version that is smaller (50K rows) and dirtier to help practice data cleaning.

The aim of this project is to clean the data and analyze the included used car listings.

## Importing data

I'll start by setting up the working environment first

In [5]:
#Importing libraries

import numpy as np
import pandas as pd

# Loading CSV to file

autos = pd.read_csv('car_data/autos.csv', encoding='Latin-1')

In [ ]:
# Quick exploration

autos.info()
print()
print(autos.head())
print()
autos.describe(include='all')

From this quick scan, a few things become apparent:
* Most columns contain string types, but some (eg. odometer, price) could probably be integers or floats, and some (eg. damage) could probably be boolean
* The data looks mostly complete, with a few columns that have some null data but nothing that stands out too much
* The 'seller' and 'offerType' column have every row except one containing the same values, could be candidates to get dropped
* There seem to be some errors in the yearOfRegistration column; the minimum is too early and the max is too high, but most values seem right
* The power also seems too high for the max level
* A "0" month of registration doesn't make sense given there's a 12'th month of registraton as well; will need to dig into this error
* There are some 4-digit postal codes; need to dig into if that makes sense
* The number of pictures is 0 across the board? Should dig into that column as well

## Cleaning data

Starting with some basic header hygiene:

In [15]:
# Replacing columns in place w/ snakecase names
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'ps_power', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pics', 'postal',
       'last_seen']

Next, replacing data that can be integers

In [49]:
# Taking a look at what the price format typically looks like
print(autos['price'].value_counts())

# Removing dollar signs and commas to leave only numbers
autos['price'] = autos['price'].str.replace('$','').str.replace(',','').astype(int)

0        1421
500       781
1500      734
2500      643
1200      639
         ... 
5485        1
33300       1
4220        1
3485        1
18430       1
Name: price, Length: 2357, dtype: int64


In [54]:
# Doing the same for the odometer column
print(autos['odometer'].value_counts())

# Removing commas and "km" from each item
autos['odometer'] = autos['odometer'].str.replace('km','').str.replace(',','').astype(int)

# Renaming the header so that the information doesn't get lost
autos.rename(columns={'odometer':'odometer_km'}, inplace=True)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64


Next, digging into the seller and offer_type columns to see if those columns can safely be dropped

In [ ]:
# Exploring some of the things mentioned above

print(autos['seller'].value_counts())
print()
print(autos['offer_type'].value_counts())
print()
print(autos[autos['seller'] != 'privat'])
print()
print(autos[autos['offer_type'] != 'Angebot'])

In [ ]:
# It's unclear why these are different (don't appear to be obious errors in any way); I'll drop these columns.
autos = autos.drop(columns=['seller','offer_type'])

In [69]:
# Checking into the other one as well..
autos['num_pics'].value_counts()
autos = autos.drop(columns='num_pics')

Now, exploring and cleaning up the two colums wer started with: odometer and price

In [ ]:
# Checking the odometer column
print(autos['price'].value_counts().sort_index())
print()

# That one seems fine; checking the price column
print(autos['price'].value_counts().sort_index())

# Digging into the maximums a bit...
print(autos['price'].value_counts().sort_index().tail(30))

# The numbers seem to jump up radically, and look 'fake' all of a sudden after the 350k mark; given how few cars are up there, we'll drop them
# We'll also drop the 0s, since I'm pretty sure eBay doesn't let you list at a 0 price at all.
autos = autos[autos['price'].between(1,350001)]

We also know we have some cleaning to do with the registration year, so let's get that done.

In [106]:
print(autos["registration_year"].describe())
print()

# Checking in on these values
print(autos[autos['registration_year'] < 1900]['registration_year'].value_counts().sort_index())
print()

# To figure out the right max, I should see when the latest add was created
print(autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index())

# None of the cars can be from later than 2016, since the latest ad was created then. This way, I know which rows to drop and can do so...
autos = autos[autos['registration_year'].between(1900,2016)]

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

1000    1
1001    1
1111    1
1800    2
Name: registration_year, dtype: int64

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64


## Analyzing data

I can now take a look at this data for some basic analysis. I'll start by looking into the brands

In [109]:
print(autos['brand'].value_counts())

volkswagen        9862
bmw               5137
opel              5022
mercedes_benz     4503
audi              4041
ford              3263
renault           2201
peugeot           1393
fiat              1197
seat               853
skoda              766
nissan             713
mazda              709
smart              661
citroen            654
toyota             593
hyundai            468
sonstige_autos     458
volvo              427
mini               409
mitsubishi         384
honda              366
kia                330
alfa_romeo         310
porsche            286
suzuki             277
chevrolet          266
chrysler           164
dacia              123
daihatsu           117
jeep               106
subaru             100
land_rover          98
saab                77
jaguar              73
daewoo              70
trabant             65
rover               62
lancia              50
lada                27
Name: brand, dtype: int64


It makes sense that German brands as well represented. Some of the other columns are in German, so it's likely that this data is an export from that geography. For the sake of learning, let's try to figure out which brand has the highest mean price.

In [ ]:
# Creating a dictionary to hold the data
brand_subset = []
brand_price = {}

# Figuring out the top 30 brands by frequency (more common) and adding them to the dictionary
brand_subset = autos['brand'].value_counts().head(30).index

# Looping over each one and figuring out the mean price
for each in brand_subset:
    brandlist = autos[autos['brand'] == each]
    meanprice = brandlist['price'].mean()
    brand_price[each] = round(meanprice,2)
    
# Printing out the top 10 most expensive (on average) brands
print(brand_price)
print()
sorted_brands = sorted(brand_price.items() ,  key=lambda x: x[1])
for elem in sorted_brands:
    print(elem[0] , ":" , elem[1] )

We can see that there's a pretty big gap between Porsche and everyone else.